<a href="https://colab.research.google.com/github/Klarret05/SenalesySistemas/blob/main/Parcial_1_SyS_2025_1_Klarret_Santiago_Castro_Castillo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Parcial 1: Conceptos Básicos y Serie de Fourier***
# ***C.C. 1090273398***
***Klarret Santiago Castro Castillo***
---



## **Punto 1**

**Punto 1:** Se tiene un microprocesador de 5 bits con entrada análoga de -3.3 a 5 [V]. Diseñe el sistema de acondicionamiento y digitalización para la señal:

\begin{equation}
x(t) = 20\sin\left(7t - \frac{\pi}{2}\right) - 3\cos(5t) + 2\cos(10t)
\end{equation}

Presente las simulaciones y gráficas de los procedimientos más representativos en un cuaderno de Python, incluyendo al menos dos períodos de la señal estudiada.

In [ ]:
'''Importar las bibliotecas necesarias'''
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist

In [ ]:
'''Funciones necesarias'''
#La función my_ceropen o cero pendiente, se encarga de ajustar la señal, es un paso condicional en donde el rango de la señal de entrada, es mapeada a un rango de salida diferente.
def my_ceropen(x,ymin = -3.3, ymax = 5): #Insertamos nuestro rango de entrada análoga (-3.3 a 5 [V]), como ymin, ymax, respectivamente.
  xmax = 25
  xmin = -25 # El valor de xmax y xmin, sale de la suma de amplitudes de las ondas de la señal dada, 20+3+2=25
  m = (ymax - ymin)/(xmax - xmin) #Se calcula la pendiente
  b = ymin - m*xmin #Y el intercepto
  yv = m*x + b #Por ultimo, se calcula la señal ajustada, con el cero pendiente, y la retorna.
  return yv

#La función de my_cuantizador, ejecuta un proceso de cuantización, en este proceso realiza un mapeo de un rango continuo de valores a una serie de valores discretos, conocidos como estados o niveles de cuantización.
def my_cuantizador(yn, vq) : #yn será un punto en el plano, el cual necesita ser cuantizado. En cambio, vq será el vector de los estados cuantizados.

  Ne = vq.shape[0] #tamaño vector de estados
  dn = cdist(yn.reshape(-1,1),vq.reshape(-1,1))#distancia yn (punto en el plano) al vector de estados cuantizados vq, la función .reshape(-1,1) asegura vectores columna para poder utilizar la biblioteca importada, cdist.
  ind = np.argmin(dn) #posición distancia, entre más pequeña sea, el estado de cuantización en vq está cerca del valor de entrada yn.
  return vq[ind] #.armin sirve para encontrar cual de lo niveles de cuantización es el más cercano al punto de valor de entrada.


In [ ]:
'''Cero y Pendiente, velocidad angular, periodo, frecuencia'''
A1 = 20 # amplitud 1ra señal - seno
A2 = -3 # amplitud 2da señal - coseno
A3 = 2 # amplitud 3ra señal - coseno

#Para el paso de seno a coseno de la primera señal, se aplica la ecuación: sin(wt+90)=-coswt, por lo tanto, la señal queda -cos(7t-(pi/2)+pi/2)=-cos(7t)

T_1 = (2/7)*np.pi # Cos 1
T_2 = (2/5)*np.pi # Cos 2
T_3 = (2/10)*np.pi # Cos 3

#Luego de haber sacado los periodos fundamentales de cada señal, se busca la frecuencia angular. Sabiendo que w= 2pi/T

w1=7
w2=5
w3=10

#Despues, teniendo velocidad angular y periodo, se halla la frecuencia de cada una. F=1/T, siendo esta el inverso de su periodo.

f1=1/T_1
f2=1/T_2
f3=1/T_3

#Para establecer los intervalos del tiempo, se es necesario tener en cuenta que el tiempo final, será igual al producto entre el periodo más grande y 4.

ti = 0 #tiempo inicial
tf = 4*T_2 #tiempo final
Fs = 36 # Por teorema de Nyquist, la frecuencia debe ser como minimo 2 o más veces la frecuencia máxima de las 3 señales. (Fs>=2Fmax)
Ts = 1/Fs
tv = np.arange(ti,tf,Ts) #vector de datos en tiempo #discretizar
y = A1*np.sin(w1*tv-np.pi/2)+ A2*np.cos(w2*tv)+ A3*np.cos(w3*tv) #vector de datos en amplitud

#cero y pendiente
ymin = -3.3
ymax = 5
nbits = 5 #Número de bits
plt.plot(y,label='y(t)') #continua

In [ ]:
ycs = my_ceropen(y, ymin,ymax)#recta aplicada con cero y pendiente
ve = np.linspace(ymin,ymax,2**nbits)#Vector de cuantización de cada uno de los estados.

#Este ciclo ejecutara de la forma de que cada una de las muestras de la recta ycs, encuentre el punto proximo al vector de cuantización, es decir, los estados, hasta que recorra los 32 estados.
for i in range(ycs.shape[0]):
   if i==0: #primera muestra
     yq = my_cuantizador(ycs[i],ve)
   else:
     yq = np.append(yq,my_cuantizador(ycs[i],ve)) #apilar datos cuantizando si i>0

#Este ciclo trazara una linea horizontal a lo largo del vector del tiempo tv en dicha amplitud.
for v in ve:
  plt.plot(tv,v*np.ones(len(tv)))#estados de cuantización

plt.stem(tv,yq , linefmt="b", markerfmt="go", basefmt="", label="Señal Discretizada")
plt.plot(tv,ycs,label='ycp(t)') #discreta cero y pendiente
plt.plot(tv,yq,c='y',marker='o',label='ydig', linewidth=2) #cuantizada + discreta = digital
plt.legend()
plt.grid()
plt.xlabel('t')
plt.ylabel('Amplitud')
plt.show()
print("Señal discretizada")

## **Punto 2**

**Punto 2:** Cuál es la señal obtenida en tiempo discreto al utilizar un conversor análogo digital con frecuencia de muestreo de 5kHz, aplicado a la señal

\begin{equation}
x(t) = 3\cos\left(100{\pi}t\right) + 5\sin(2000{\pi}t) + 10\cos(11000{\pi}t)
\end{equation}

Realizar la simulación del proceso de discretización. En caso de que la discretización no sea apropiada, diseñe e implemente un conversor adecuado para la señal estudiada.

In [ ]:
'''Importar las bibliotecas necesarias'''
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

# Definir las frecuencias de las componentes de la señal en Hertz (Hz).
f1 = 500   # Hz
f2 = 1000  # Hz
f3 = 5500  # Hz

# Establecer una frecuencia de muestreo muy alta para simular una señal continua.
fs_high = 15000  # Frecuencia de muestreo alta para simular señal "continua"

# Crear un vector de tiempo para la señal de alta frecuencia. Se elige un intervalo corto (0.01s) para visualización.
t_high = np.arange(0, 0.01, 1/fs_high)  # Vector de tiempo para la señal "continua"

# Generar la señal original sumando tres componentes sinusoidales con sus respectivas amplitudes y frecuencias.
x_high = (3 * np.cos(2 * np.pi * f1 * t_high) +
          5 * np.sin(2 * np.pi * f2 * t_high) +
          10 * np.cos(2 * np.pi * f3 * t_high))

# --- Diseño e implementación del filtro antialiasing ---

# Calcular la frecuencia de Nyquist, que es la mitad de la frecuencia de muestreo de la señal "continua".
nyquist = 0.5 * fs_high

# Definir la frecuencia de corte para el filtro paso bajo.
cutoff = 2500  # Frecuencia de corte del filtro antialiasing

# Definir el orden del filtro. Un orden mayor generalmente resulta en una transición más pronunciada.
order = 6

# Normalizar la frecuencia de corte con respecto a la frecuencia de Nyquist.
normal_cutoff = cutoff / nyquist

# Diseñar el filtro Butterworth de paso bajo utilizando los parámetros definidos.
# La función devuelve los coeficientes del numerador (b) y del denominador (a) del filtro.
b, a = signal.butter(order, normal_cutoff, btype='low')

# Aplicar el filtro a la señal "continua".
# signal.filtfilt aplica el filtro hacia adelante y hacia atrás para evitar desfases.
x_filtered = signal.filtfilt(b, a, x_high)

# --- Muestreo de la señal filtrada ---

# Definir la frecuencia de muestreo a la que se discretizará la señal.
fs_low = 5000  # Frecuencia de muestreo deseada (5 kHz)

# Calcular el factor de diezmado (cuántas muestras de la señal de alta frecuencia se tomarán para obtener la señal de baja frecuencia).
factor = int(fs_high / fs_low)

# Muestrear la señal filtrada tomando una muestra cada 'factor' muestras.
x_filtered_low = x_filtered[::factor]

# Obtener el vector de tiempo correspondiente a las muestras discretas.
t_low = t_high[::factor]

# --- Cálculo de las Transformadas de Fourier ---

# Calcular la FFT de la señal original de alta frecuencia.
fft_high = np.fft.rfft(x_high)

# Calcular las frecuencias correspondientes a los puntos de la FFT de alta frecuencia.
freqs_high = np.fft.rfftfreq(len(x_high), 1/fs_high)

# Calcular la FFT de la señal discretizada de baja frecuencia.
fft_filtered = np.fft.rfft(x_filtered_low)

# Calcular las frecuencias correspondientes a los puntos de la FFT de baja frecuencia.
freqs_low = np.fft.rfftfreq(len(x_filtered_low), 1/fs_low)

# --- Graficar los resultados en el dominio del tiempo y la frecuencia ---

# Crear una figura con un tamaño específico para las subplots.
plt.figure(figsize=(12, 10))

# Subplot 1: Señal original en el dominio del tiempo.
plt.subplot(3, 2, 1)
plt.plot(t_high, x_high, label='Original')
plt.title('Señal original (Dominio del tiempo)')
plt.xlabel('Tiempo [s]')
plt.xlim(0, 0.005)  # Limitar el eje x para mostrar solo los primeros 5 ms

# Subplot 2: Espectro de la señal original (magnitud).
plt.subplot(3, 2, 2)
plt.plot(freqs_high, np.abs(fft_high))
plt.title('Espectro original')
plt.xlabel('Frecuencia [Hz]')
plt.xlim(0, 6000) # Limitar el eje x para visualizar las componentes principales

# Subplot 3: Señal filtrada en el dominio del tiempo.
plt.subplot(3, 2, 3)
plt.plot(t_high, x_filtered, 'red', label='Filtrada')
plt.title('Señal después del filtro antialiasing')
plt.xlabel('Tiempo [s]')
plt.xlim(0, 0.005) # Limitar el eje x

# Subplot 4: Espectro de la señal filtrada (magnitud).
plt.subplot(3, 2, 4)
plt.plot(freqs_high, np.abs(np.fft.rfft(x_filtered)), color='r')
plt.title('Espectro después del filtro')
plt.xlabel('Frecuencia [Hz]')
plt.xlim(0, 6000) # Limitar el eje x

# Subplot 5: Señal discretizada en el dominio del tiempo (utilizando un stem plot para mostrar muestras discretas).
plt.subplot(3, 2, 5)
plt.stem(t_low, x_filtered_low, linefmt='C3-', markerfmt='C3o', basefmt='C3-')
plt.title('Señal discretizada (5 kHz)')
plt.xlabel('Tiempo [s]')
plt.xlim(0, 0.005) # Limitar el eje x

# Subplot 6: Espectro de la señal discretizada (magnitud).
plt.subplot(3, 2, 6)
plt.plot(freqs_low, np.abs(fft_filtered), color='red')
plt.title('Espectro discretizado')
plt.xlabel('Frecuencia [Hz]')
plt.xlim(0, 2500) # Limitar el eje x al rango de frecuencias representable (Nyquist de la señal discretizada)

# Ajustar el espaciado entre subplots para evitar superposiciones.
plt.tight_layout()

# Mostrar las gráficas.
plt.show()

# --- Mostrar resultados numéricos y conclusiones ---

# Imprimir las primeras 10 muestras de la señal discretizada para inspección.
print("\nPrimeras 10 muestras de la señal discretizada:")
print(np.round(x_filtered_low[:10], 4)) # Redondear a 4 decimales para mejor legibilidad

# Presentar una conclusión sobre el proceso y el efecto del filtro antialiasing.
print("Conclusión:")
print("- La frecuencia de muestreo de 5 kHz no es adecuada para la componente de 5500 Hz (aparece aliasing a 500 Hz).")
print("- Se implementó un filtro antialiasing de 2500 Hz para eliminar frecuencias altas.")
print("- La señal discretizada correctamente contiene solo las componentes de 500 Hz y 1000 Hz.")

## **Punto 3**

**Punto 3:** La distancia media entre dos señales \begin{equation}
x_1(t), x_2(t) \in \mathbb{R},\, \mathbb{C}
\end{equation} , se puede expresar a partit de la potencia media:

\begin{equation}
d(x_1, x_2) = \overline{P}_{x_1 - x_2} = \lim_{T \to \infty} \frac{1}{T} \int_{0}^{T} |x_1(t) - x_2(t)|^2 \, dt.
\end{equation}

Sean \begin{equation}
x_1(t), x_2(t):
\end{equation}

\begin{equation}
x_1(t) = A \cos(w_0 t), \quad w_0 = \frac{2\pi}{T}, \quad T, A \in \mathbb{R}^+
\end{equation}

\begin{equation}
x_2(t) =
\begin{cases}
1  & \text{si } 0 \leq l < \frac{T}{4} \\
-1 & \text{si } \frac{T}{4} \leq l < \frac{3T}{4} \\
1  & \text{si } \frac{3T}{4} \leq l < T
\end{cases}
\end{equation}

¿Cuál es la distancia media entre las señales?. Corrobore sus desarrollos con Sympy.


In [ ]:
'''Importar las bibliotecas necesarias'''
import sympy as sp

# Se definen las variables simbolicas a utilizar.
A,t, T = sp.symbols('A t T', real=True, positive=True)
w0 = 2 * sp.pi / T
# Definición de las señales
x1 = A*sp.cos(w0 * t)

# Definición por partes de ambas funciones, esto mediante .piecewise,que permite que una función tenga distintos valores en diferentes intervalos de t.
x2 = sp.Piecewise(
    (1, (t >= 0) & (t < T/4)),
    (-1, (t >= T/4) & (t < 3*T/4)),
    (1, (t >= 3*T/4) & (t < T))
)

# Función a integrar se define como f: (calcula la diferencia al cuadrado entre las dos señales x1, x2)
f = (x1 - x2)**2

# Se emplea la integral tramo por tramo, para al final sumarlas.
integral = sp.integrate(f, (t, 0, T/4)) + sp.integrate(f, (t, T/4, 3*T/4)) + sp.integrate(f, (t, 3*T/4, T))

# Para este punto se tiene en cuenta que la distancia se define como la división de la integral dada entre el periodo.
distancia_media = integral / T

# Simplificación de la expresión de la distancia
distancia_media_simplificada = sp.simplify(distancia_media)

distancia_media_simplificada

## **Punto 4**

**Punto 4:** Sea x''(x) la segunda derivada de la señal x(t), donde t    - Duemuestre que los coeficientes de la serie exponencial de Fourier se pueden calcular según:

\
\begin{align*}
c_n = \frac{1}{(t_i - t_f) n^2 w_o^2} \int_{t_i}^{t_f} x''(t) e^{-j n w_o t} \, dt; \quad n \in \mathbb{Z}.
\end{align*}

¿Cómo se pueden calcular los coeficientes an y bn desde en la serie trigonométrica de Fourier?.

Encuentre el espectro de Fourier, su magnitud, fase, parte real, parte imaginaria y el error relativo de reconstrucción para \begin{equation}
n \in \{0, \pm 1, \pm 2, \pm 3, \pm 4, \pm 5\} \end{equation}
, a partir de x''(t) para la señal x(t) en la Figura 1. Compruebe el espectro obtenido con la estimación a partir de x(t) y presente las respectivas simulaciones sobre Python.


In [ ]:
'''Importar las bibliotecas necesarias'''
from ipywidgets import interact,IntSlider

#Parametros  relacionados a la forma y escala de la señal a generar
A=1
d2=2
d1=1

#Generar señal parametrizada en función del tiempo
T = 2*d2 # Duración total de la señal para ser graficada
Fo = 1/T # Frecuencia fundamental de la señal
Fs = 1000*Fo # Frecuencia de muestreo
tv = np.arange(-T/2,T/2,1/Fs) #vector de tiempo, el cual esta formado valores espaciados en secuencia dentro del intervalo.
Nm = len(tv) #Número de estados de cuantización.

#Genera valores basados en la figura geometrica del trapecio, siendo x, la variable con la que esta trabajara, y a,b,c,d serán los que definiran la forma del mismo.
def funcion_trapecio(x, a, b, c, d):
    """

    Parámetros:
x: Un arreglo de NumPy con los valores del eje x.
a: La coordenada x donde comienza la pendiente ascendente del trapecio.
b: La coordenada x donde comienza la parte plana (meseta) del trapecio.
c: La coordenada x donde termina la parte plana (meseta) del trapecio.
d: La coordenada x donde termina la pendiente descendente del trapecio.
    """
    return np.maximum(0, np.minimum((x - a) / (b - a), np.minimum(1, (d - x) / (d - c))))

# Ejemplo de uso
x = funcion_trapecio(tv, -d2, -d1, d1, d2)



#graficar señal original
plt.plot(tv,x,'b',linewidth=4)
plt.grid()
plt.xlabel("t[s]",fontsize = 14)
plt.ylabel("x(t)",fontsize = 14)
plt.show()

In [ ]:
N = 5 #numero armonicos
wo = 2*np.pi/T # frecuencia fundamental
# Creación de bases, funciones de base para las series exponenciales de Fourier, en esta se usaran exponenciales complejos como base.
phin = np.zeros((Nm,2*N+1),np.complex128)#arreglo de ceros que guardara las bases exponenciales complejas.
for n in range(-N,N+1,1):
    phin[:,n+N] = np.exp(1j*n*wo*tv)#base de fourier en el intervalo de interés, va ir con el armonico de -a hacia +a. Donde calculara el exponencial complejo de cada armonico n.

#Calcula el espectro,pero con los coeficientes de series de Fourier basados en la señal del trapecio hecha previamente.
cn = np.zeros(2*N+1,np.complex128)
nv = np.linspace(-N, N, 2*N+1) #Vector de armonicos
Cn = (-2*(np.cos(nv*np.pi)-np.cos(nv*np.pi*0.5)))/(np.pi**2*nv**2)
Cn[N] = 3/4 #Nivel DC (Componente real)

In [ ]:
#graficar espectro
fig = plt.figure(figsize=(8,8))
plt.subplot(2,2,1)
plt.stem(nv, np.real(Cn), 'r')
plt.xlabel(r'$nw_o$[rad/s]',fontsize = 14)
plt.ylabel(r'$Re\{C_n\}$',fontsize = 14)
plt.axis('tight')
plt.grid()

plt.subplot(2,2,2)
plt.stem(nv, np.imag(Cn), 'r')
plt.xlabel(r'$nw_o$[rad/s]',fontsize = 14)
plt.ylabel(r'$Im\{C_n\}$',fontsize = 14)
plt.axis('tight')
plt.grid()

plt.subplot(2,2,3)
plt.stem(nv, abs(Cn), 'r')#magnitud del espectro
plt.xlabel(r'$nw_o$[rad/s]',fontsize = 14)
plt.ylabel(r'$|C_n|$',fontsize = 14)
plt.axis('tight')
plt.grid()

plt.subplot(2,2,4)
plt.stem(nv, np.angle(Cn), 'r')#fase del espectro
plt.xlabel(r'$nw_o$[rad/s]',fontsize = 14)
plt.ylabel(r'$\langle C_n$',fontsize = 14)
plt.axis('tight')
plt.grid()

fig.tight_layout()

In [ ]:
Px = 2/3 # Potencia de la señal original

# La función pltest, ayuda a la reconstrucción y grafico de la señal
def pltest(Na=1):
    ind = np.arange(N-Na,N+Na+1)
    er = 1- np.sum(abs(Cn[ind])**2)/Px
    #señal reconstruida
    xe = phin[:,ind].dot(Cn[ind] ) # dot calcula el producto entre vectores
    plt.plot(tv,xe,color='b',label='$x_e(t)$') # señal estimada o filtrada
    plt.plot(tv,x,color='g',label='x(t)') #señal original
    plt.title('$E_r$=%.2f[%%]' % (100*er),fontsize=16)
    plt.xlabel('t[s]')
    plt.ylabel('x(t)')
    plt.grid()
    plt.legend()
    plt.show()

    return
interact(pltest,Na=(1,N,1))